In [1]:
import numpy as np
import six
import sys
import os
import traceback
import re
import pickle
from copy import deepcopy

from chainer import cuda
from context2vec.common.context_models import Toks
from context2vec.common.model_reader import ModelReader
import sklearn
import pandas as pd
import logging
from scipy.stats import spearmanr
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import gensim
import math




[nltk_data] Downloading package stopwords to /home/ql261/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def produce_top_n_simwords(w_filter,context_embed,n_result,index2word):
        #assume that w_filter is already normalized
        context_embed = context_embed / xp.sqrt((context_embed * context_embed).sum())
        similarity_scores=[]
        print('producing top {0} simwords'.format(n_result))
        similarity = (w_filter.dot(context_embed)+1.0)/2
        top_words_i=[]
        top_words=[]
        count = 0
        for i in (-similarity).argsort():
                    if xp.isnan(similarity[i]):
                        continue
                    print('{0}: {1}'.format(str(index2word[i]), str(similarity[i])))
                    count += 1
                    top_words_i.append(i)
                    top_words.append(index2word[i])
                    similarity_scores.append(similarity[i])
                    if count == n_result:
                        break

        top_vec=w_filter[top_words_i,:]
        
        return top_vec,np.array(similarity_scores),top_words
    
def top_mutual_sim(top_vec,similarity_scores):

    #normalize the top_vec
    s = np.sqrt((top_vec * top_vec).sum(1))
    s[s==0.] = 1.
    top_vec /= s.reshape((s.shape[0], 1))
    
    # substitutes' similarity to sentence (similarity_scores) as weight matrix to mutual similarity
    max_score=similarity_scores[0]
    similarity_scores=np.array(similarity_scores)
    sim_weights=(similarity_scores+similarity_scores.reshape(len(similarity_scores),1))/2.0
    #weighted by the maximum score in the substitutes (highre max score means the context is more certain about the substitutes)
    sim_weights=(sim_weights/float(sum(sum(sim_weights))))*max_score
    # dot product weighted by substitute probability (sim_weights)
    inf_score=sum(sum(top_vec.dot(top_vec.T)*sim_weights))
    return inf_score

def top_cluster_density(top_vec,similarity_scores):
    #normalize the top_vec
    s = np.sqrt((top_vec * top_vec).sum(1))
    s[s==0.] = 1.
    top_vec = top_vec/ s.reshape((s.shape[0], 1))
    
    #perform the centroid
    max_score=similarity_scores[0]
    similarity_scores=np.array(similarity_scores).reshape(len(similarity_scores),1)/sum(similarity_scores)
    centroid_vector=sum(top_vec*similarity_scores)
    # average of cosine distance to the centroid,weighted by max scores
    inf_score=sum(top_vec.dot(centroid_vector))/len(top_vec)*max_score
    return inf_score

In [3]:
top_cluster_density(np.array([[1.,1.,1.],[4.,4.,4.]]),np.array([0.3,0.1]))
# np.linalg.norm([0.57735027,0.57735027, 0.57735027])

0.3

In [20]:
def load_w2salience(w2salience_f,weight_type):
    w2salience={}
    with open(w2salience_f) as f:
        for line in f:
            if line.strip()=='':
                continue
            w,w_count,s_count=line.strip().split('\t')
            if weight_type==INVERSE_W_FREQ:
                w2salience[w]=1/float(w_count)
            elif weight_type==INVERSE_S_FREQ:
                w2salience[w]=math.log(1+84755431/float(s_count))
    return w2salience

def skipgram_context(model,words,pos,weight=None,w2entropy=None):
    context_wvs=[]
    weights=[]
    for i,word in enumerate(words):
        if i != pos: #surroudn context words
            try:
                if weight ==LDA:
                    if word in w2entropy and word in model:
                        print (word,w2entropy[word])
                        weights.append(1/(w2entropy[word]+1.0))
                        context_wvs.append(model[word])
                elif weight in [INVERSE_W_FREQ,INVERSE_S_FREQ]:
                    if word in w2entropy and word in model:
                        print (word,w2entropy[word])
                        weights.append(w2entropy[word])
                        context_wvs.append(model[word])
                else:
                    #equal weights per word
                    context_wvs.append(model[word])
                    weights.append(1.0)
            except KeyError as e:
                print ('==warning==: key error in context {0}'.format(e))
    context_embed=sum(np.array(context_wvs)*np.array(weights).reshape(len(weights),1))#/sum(weights)
    return sum(weights),context_embed #  will be normalized later

def lg_model_out_w2v(top_words,w_target,word2index_target):
        # lg model substitutes in skipgram embedding
        top_vec=[]
        index_list=[]
        for i,word in enumerate(top_words):
            try :
                top_vec.append(w_target[word2index_target[word]])
                index_list.append(i)
            except KeyError as e:
                print (e)
        return np.array(top_vec),index_list
    
def context_inform(test_s,test_w, model,model_type,n_result,w_filter,index2word,weight,w2entropy=None,w_target=None,word2index_target=None,index2word_target=None):
    #produce context representation and infromative score for each context
    test_s=test_s.replace(test_w, ' '+test_w+' ')
    print(test_s)
    words=test_s.split()
    pos=words.index(test_w)
    
    score=1.0 #default score
    
    # Decide on the model
    if model_type=='context2vec':
        context_embed= model.context2vec(words, pos)
        context_embed_out=context_embed
    
    elif model_type=='skipgram':
        score,context_embed=skipgram_context(model,words,pos,weight,w2entropy)
        context_embed_out=context_embed
        
    elif model_type=='context2vec-skipgram':
        # context2vec substitutes in skipgram space
        context_embed= model.context2vec(words, pos)
        top_vec,sim_scores,top_words=produce_top_n_simwords(w_filter,context_embed,n_result,index2word)
        top_vec,index_list=lg_model_out_w2v(top_words,w_target,word2index_target) 
        sim_scores=sim_scores[index_list] #weighted by substitute probability
        context_embed_out=sum(top_vec*((sim_scores/sum(sim_scores)).reshape(len(sim_scores),1)))
    else:
        print ('model type {0} not recognized'.format(model_type))
        sys.exit(1)
        
        
    #decide on weight per sentence
    if weight==TOP_MUTUAL_SIM:
        print (weight)
#         if word2index_target==None: #not context2vec-skipgram
#             context2vec word embedding space neighbours
        top_vec,sim_scores,top_words=produce_top_n_simwords(w_filter,context_embed,n_result,index2word)
        #skipgram word embedding space neighbours when context2vec-skipgram
        score=top_mutual_sim(top_vec,sim_scores)
        print (score)
    elif weight==TOP_CLUSTER_DENSITY:
        print (weight)
#         if word2index_target==None: #not context2vec-skipgram
#             context2vec word embedding space neighbours
        top_vec,sim_scores,top_words=produce_top_n_simwords(w_filter,context_embed,n_result,index2word)
        score=top_cluster_density(top_vec,sim_scores)
        print (score)
        
    elif weight=='learned':
        print ('learned not implemented')
    elif weight=='gaussian':
        print ('gaussian not implemented')
    elif weight ==False or weight in [LDA,INVERSE_S_FREQ,INVERSE_W_FREQ]:
        score=score
    else:
        print ('weight mode {0} not recognized'.format(weight))
    return score,context_embed_out

def additive_model(f_w,test_ss,test_w, model_type,model,n_result,w_filter,index2word,weight=False,w2entropy=None,w_target=None,word2index_target=None,index2word_target=None):
    #produce context representation across contexts using weighted average
    
    context_out=[]
    context_weights=[]
    for test_s in test_ss.split('@@'):
        test_s=test_s.strip()
        #produce context representation with scores
        score,context_embed=context_inform(test_s,test_w, model,model_type,n_result,w_filter,index2word,weight,w2entropy,w_target,word2index_target,index2word_target)
        print ('weight is {0}'.format(score))
        context_out.append(context_embed)
        context_weights.append(score)
    
    
    #sum representation across contexts
    context_out=np.array(context_out)
    norm_weights=np.array(context_weights).reshape(len(context_weights),1)/float(sum(context_weights))
    f_w.write(','.join([str(i[0]) for i in norm_weights])+'\n')
    print ('normalized weight: \n  {0}'.format(norm_weights))
    
    if model_type=='skipgram':
        # context representation by weighted sum of all context words in all contexts
        context_avg=sum(context_out)/sum(context_weights)
    else:
        # context represenatation by weighted sum of contexts
        context_avg=sum(norm_weights*context_out)
    
    
    # check new embedding neighbours

    print('producing top {0} words for new embedding'.format(n_result))
    if index2word_target==None:
        top_vec,scores,top_words=produce_top_n_simwords(w,context_avg,n_result,index2word)
    else:
        #print the target space neighbours for context2vec-skipgram
        print (w_target.shape)
        top_vec,scores,top_words=produce_top_n_simwords(w_target,context_avg,n_result,index2word_target)
    
    return context_avg




In [5]:
def filter_w(w,word2index,index2word):
    #filter out words with no letters in, and stopwords
    stopw=stopwords.words('english')
    stopw=[word.encode('utf-8') for word in stopw]
    index2word_filter={}
    word2index_filter={}
    index_filter2index=[]
    counter=0
    for word in word2index:
            if word not in stopw:
                    index_filter2index.append(word2index[word])
                    word2index_filter[word]=counter
                    index2word_filter[counter]=word
                    counter+=1
    w_filter= w[index_filter2index,:]
    return w_filter,word2index_filter,index2word_filter

def rm_stopw_context(model):
    stopw=stopwords.words('english')
    stopw=[word.encode('utf-8') for word in stopw]
    
    model={word:model.wv.__getitem__(word) for word in model.wv.vocab if word not in stopw}
    return model




In [6]:
def eval_chimera(chimeras_data_f,context_model,model_type,n_result,w,index2word,weight=False,w2entropy=None,w_target=None,word2index_target=None,index2word_target=None):
    chimeras_data_dir='/'.join(chimeras_data_f.split('/')[:-1])
    num_sent=chimeras_data_f.split('/')[-1].split('.')[1][1]
    print (chimeras_data_dir)
    print (num_sent)
    with open(chimeras_data_dir+'/weights_{0}_{1}_{2}'.format(num_sent,model_type,str(weight)),'w') as f_w:
        spearmans=[]
        data=pd.read_csv(os.path.join(chimeras_data_f),delimiter='\t',header=None)

        for index, row in data.iterrows():
            golds=[]
            model_predict=[]
            probes=[]
            #compute context representation
            if weight!='learned':
                context_avg=additive_model(f_w,row[1].lower(),'___', model_type,context_model,n_result,w,index2word,weight,w2entropy,w_target,word2index_target,index2word_target)
            context_avg = context_avg / xp.sqrt((context_avg * context_avg).sum())

            #cosine similarity with probe embedding
            for gold,probe in zip(row[3].split(','),row[2].split(',')):
                try:
                    if index2word_target==None:
                        probe_w_vec=xp.array(w[word2index[probe]])
                    else:
                        probe_w_vec=xp.array(w_target[word2index_target[probe]])
                    probe_w_vec=probe_w_vec/xp.sqrt((probe_w_vec*probe_w_vec).sum())
                    cos=probe_w_vec.dot(context_avg)
                    if xp.isnan(cos):
                        continue
                    else:
                        model_predict.append(cos)
                        golds.append(gold)
                        probes.append(probe)
                except KeyError as e:
                    print ("====warning key error for probe=====: {0}".format(e))
            print ('probes',probes)
            print ('gold',golds)
            print ('model_predict',model_predict)
            sp=spearmanr(golds,model_predict)[0]
            print ('spearman correlation is {0}'.format(sp))
            if not math.isnan(sp):
                spearmans.append(sp)
        print ("AVERAGE RHO:",float(sum(spearmans))/float(len(spearmans)))

In [13]:
TOP_MUTUAL_SIM='top_mutual_sim'
TOP_CLUSTER_DENSITY='top_cluster_density'
LDA='lda'
INVERSE_S_FREQ='inverse_s_freq'
INVERSE_W_FREQ='inverse_w_q'
WEIGHT_DICT={0:False,1:TOP_MUTUAL_SIM,2:LDA,3:INVERSE_S_FREQ,4:INVERSE_W_FREQ,5:TOP_CLUSTER_DENSITY}


if __name__=="__main__":
    
    #params read in
    if sys.argv[0]=='/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py':
        
        data='./eval_data/data-chimeras/dataset.l2.fixed.test.txt.punct'

        weight=WEIGHT_DICT[0]
        
#         ##context2vec
##         model_param_file='../models/context2vec/model_dir/context2vec.ukwac.model.params'
#         model_param_file='../models/context2vec/model_dir/MODEL-wiki.params.14'
        
#         model_type='context2vec'
#         context_rm_stopw=0

# ####skipgram
#         model_param_file='../models/wiki_all.model/wiki_all.sent.split.model'
#         model_type='skipgram'
#         context_rm_stopw=1
#         weight='inverse_w_freq'
#         w2salience_f='../corpora/corpora/wiki.all.utf8.sent.split.tokenized.vocab'
#         w2salience_f='../models/lda/w2entropy'


####context2vec-skipgram
        model_param_file='../models/context2vec/model_dir/MODEL-wiki.params.14?../models/wiki_all.model/wiki_all.sent.split.model'
#         model_param_file='../models/context2vec/model_dir/context2vec.ukwac.model.params?../models/wiki_all.model/wiki_all.sent.split.model'
        model_type='context2vec-skipgram'
#         context_rm_stopw=0
        n_result=20
        w2salience_f=None
    
    else:
        if len(sys.argv) < 5:
            print >> sys.stderr, "Usage: {0} <model_param_file> <model_type> <weight:{1}> <eval_data> <w2salience>"  .format (sys.argv[0],WEIGHT_DICT.items())
            sys.exit(1)
        
        model_param_file = sys.argv[1]
        model_type=sys.argv[2]
        
        if '-' in sys.argv[3]:
            weight,n_result=sys.argv[3].split('-')
            weight=WEIGHT_DICT[int(weight)]
            n_result=int(n_result)
        else:
            weight=WEIGHT_DICT[int(sys.argv[3])]
            n_result=20 #default is 20 top
            
#         context_rm_stopw=int(sys.argv[4])
        data =sys.argv[4]
        
        if len(sys.argv)>5:
            w2salience_f=argv[5]
        else:
            w2salience_f=None
    
    #gpu setup 
    gpu = -1 # todo: make this work with gpu

    if gpu >= 0:
        cuda.check_cuda_available()
        cuda.get_device(gpu).use()    
    xp = cuda.cupy if gpu >= 0 else np
    
    # logging
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)

    
    #choose model type
    print ('read model....')
    if model_type=='context2vec':
        #read in model
        
        model_reader = ModelReader(model_param_file)
        w = model_reader.w
        index2word = model_reader.index2word
        word2index=model_reader.word2index
        model = model_reader.model
        w_target=None
        word2index_target=None
        index2word_target=None
        context_rm_stopw=0
    elif model_type=='skipgram':
        model = gensim.models.Word2Vec.load(model_param_file)
        w=deepcopy(model.wv.vectors)
        #vector normalize for probe w embedding
        s = np.sqrt((w * w).sum(1))
        s[s==0.] = 1.
        w /= s.reshape((s.shape[0], 1))
        
        index2word=model.wv.index2word
        word2index={key: model.wv.vocab[key].index for key in model.wv.vocab}
        w_target=None
        word2index_target=None
        index2word_target=None
        context_rm_stopw=1
        
    elif model_type=='context2vec-skipgram':
        model_param_context,model_param_w2v=model_param_file.split('?')
        model_reader = ModelReader(model_param_context)
        w = model_reader.w
        index2word = model_reader.index2word
        word2index=model_reader.word2index
        model = model_reader.model
        
        model_w2v = gensim.models.Word2Vec.load(model_param_w2v)
        w_target=model_w2v.wv.vectors
        index2word_target=model_w2v.wv.index2word
        word2index_target={key: model_w2v.wv.vocab[key].index for key in model_w2v.wv.vocab}
        context_rm_stopw=0
    
    w2salience=None
    
    #remove stop words in target word space
    print ('filter words for target....')
    w,word2index,index2word=filter_w(w,word2index,index2word)
    if  index2word_target!=None:
        w_target,word2index_target,index2word_target=filter_w(w_target,word2index_target,index2word_target)
    
    #weight

#     if weight==TOP_MUTUAL_SIM of weight==TOP_CLUSTER_DENSITY:
    if weight==LDA:
        print ('load vectors and entropy')
        w2salience=pickle.load(open(w2salience_f))
    elif weight==INVERSE_W_FREQ:
        print ('load w2freq')
        w2salience=load_w2salience(w2salience_f,weight)
    elif weight==INVERSE_S_FREQ:
        print ('load w2freq')
        w2salience=load_w2salience(w2salience_f,weight)


    # remove context stop words
    if int(context_rm_stopw)==1:
        print ('filter words for context....')

        model=rm_stopw_context(model)
        
    
    print (model_param_file,model_type,weight,context_rm_stopw,data,w2salience_f)


read model....
Reading config file: ../models/context2vec/model_dir/MODEL-wiki.params.14
Config:  {'config_path': '../models/context2vec/model_dir/', 'model_file': 'MODEL-wiki.14', 'deep': 'yes', 'drop_ratio': '0.0', 'words_file': 'WORDS-wiki.targets.14', 'unit': '400'}
filter words for target....
('../models/context2vec/model_dir/MODEL-wiki.params.14?../models/wiki_all.model/wiki_all.sent.split.model', 'context2vec-skipgram', False, 0, './eval_data/data-chimeras/dataset.l2.fixed.test.txt.punct', None)


In [23]:
#read in data
# data='./eval_data/data-chimeras/dataset.l6.fixed.test.txt.punct'
if data.split('/')[-2]== 'data-chimeras':
#         weight=WEIGHT_DICT[1]
#         print (weight)
        eval_chimera(data,model,model_type,n_result,w,index2word,weight,w2salience,w_target,word2index_target,index2word_target)
        

./eval_data/data-chimeras
6
canned sardines and  ___  between two slices of wholemeal bread and thinly spread flora original .
producing top 20 simwords
meatloaf: 0.5400038
sandwiches: 0.5396633
tamales: 0.538706
chicken: 0.53835166
spreads: 0.53812134
popcorn: 0.5368529
sliced: 0.53673893
mash: 0.53662497
squeezed: 0.5363477
sorbet: 0.53589183
sandwiched: 0.53515226
cooked: 0.5348494
stuffed: 0.5340092
couscous: 0.53323156
fritters: 0.5331776
sauerkraut: 0.5330202
mixtures: 0.532765
alternates: 0.5324085
pickles: 0.5319029
smothered: 0.5318118
top_mutual_sim
0.19941598433069885
weight is 0.199415984331
erm ,  ___  , low fat dairy products , incidents of heart disease for those who have an olive oil rich diet .
producing top 20 simwords
dairies: 0.54627275
pharmaceuticals: 0.53947484
pharma: 0.53916913
probiotics: 0.5363117
medications: 0.5351843
freezers: 0.5344019
stocks: 0.53410417
toothpastes: 0.53363115
cigarettes: 0.53224295
antibiotics: 0.5318707
fertilizers: 0.53155047
cosmetic

marinated: 2.839213173306499
shallots: 2.779845520157031
cilantro: 2.741722557101824
coriander: 2.7239848722692352
stewed: 2.7096621341678166
chickpeas: 2.7072608491425627
salads: 2.6732456564297014
soups: 2.6536638595778252
minced: 2.6361280577324653
braised: 2.618158738640356
sauces: 2.616634596907246
mozzarella: 2.6052938700418005
flavorings: 2.596286913855234
chilies: 2.580026101876738
lentils: 2.5771742971117377
garlic: 2.5763728071065195
pickled: 2.574889568694207
turmeric: 2.5743217789216533
toppings: 2.5730604486147586
almonds: 2.56920423990936
('probes', ['rhubarb', 'onion', 'pear', 'strawberry', 'limousine', 'cushion'])
('gold', ['3', '3.86', '4', '3.71', '2.14', '2.29'])
('model_predict', [0.5565775114417657, 0.6541480522869645, 0.5186031081739167, 0.49059318621088754, 0.19132374485654413, 0.25633644442629544])
spearman correlation is 0.714285714286
here are a few suggestions: � how many of us know how much water our washing machines  ___  use ?
producing top 20 simwords
cou

publicationdate: 2.135862997008297
frass: 2.134084964080445
hindwings: 2.122508469726334
ochreous: 2.1118686021743187
editorgiven1: 2.111554411685394
editorsurname4: 2.100079669432309
phyllaries: 2.0962972318364144
1px: 2.0712683237002034
hydroxide: 2.0683032959800816
florets: 2.0656472027559034
pelecaniformesfamily: 2.058470050802227
forewings: 2.0564480385057005
discal: 2.050215888174679
rinsed: 2.0485040447630842
fluidized: 2.0435446029449387
editorsurname1: 2.039375266710746
drupe: 2.0388693764861157
brownish: 2.0349349650293
whitish: 2.0223745453700053
fuscous: 2.0213192634689996
('probes', ['stove', 'microwave', 'kettle', 'cage', 'wastebin', 'leopard'])
('gold', ['2.86', '2.57', '2.14', '1.57', '1.57', '1.71'])
('model_predict', [0.6073783985065612, 0.4155469767535527, 0.44520275444932034, 0.36741041360834037, 0.33511909720436406, 0.26143375707393224])
spearman correlation is 0.753702346348
what never needs sharpening , is completely  ___  safe and is covered by a 25 year guarant

to-surface: 2.337417674167229
aim-9: 2.2471627889455426
icbms: 2.2380491897921755
recoilless: 2.211031058376351
0px: 2.2069418200903987
laysummary: 2.202114526745725
rpg-7: 2.200415014939404
launchers: 2.1905056592368037
missiles: 2.1827772138793478
pyrams: 2.1819555558649544
apfsds: 2.163528427732339
warheads: 2.1504848813970554
cosmodrome: 2.1471396899397903
editorgiven1: 2.1469678464882374
warhead: 2.146599174496544
63sm: 2.12794775915207
autocannon: 2.122150706854388
aim-7: 2.1209547812750165
ciws: 2.1176425101419136
editorsurname4: 2.1153464478827444
('probes', ['bazooka', 'gun', 'bayonet', 'bullet', 'buckle', 'canoe'])
('gold', ['2.86', '2.57', '2.57', '2.29', '1.29', '2.57'])
('model_predict', [0.4808406661572478, 0.4507917327315308, 0.4282706451907633, 0.3930550598539022, 0.23603380154814427, 0.24963469385163034])
spearman correlation is 0.819688599971
the n 1 r ' s internal memory gives you access to an amazing 1,671 sounds + 39  ___  kits .
producing top 20 simwords
stereo: 0

bassoons: 2.49321929243474
oboes: 2.409268836763436
editorgiven1: 2.3620251472454115
editorsurname4: 2.3170125564141655
trombones: 2.267647847149532
clarinets: 2.258934910927124
laysummary: 2.247463480948989
timpani: 2.2208581332378556
laydate: 2.2098841830560314
cymbals: 2.1681041893988153
publicationdate: 2.1269486520960212
glockenspiel: 2.1264759723525617
editorsurname3: 2.1192017060101307
flutes: 2.108407832564618
contrabassoon: 2.105003954777716
editorsurname2: 2.1045965388033316
editorsurname1: 2.0930253150128233
trumpets: 2.0656473753742524
violas: 2.063531592953696
cellos: 2.060083230272939
('probes', ['bagpipe', 'harmonica', 'whistle', 'shotgun', 'bear', 'bouquet'])
('gold', ['1.71', '2', '1.43', '1.43', '1.14', '2.14'])
('model_predict', [0.4272351815139718, 0.5369120826464338, 0.4611995327934169, 0.4080507356142732, 0.22973520284958004, 0.298897594683764])
spearman correlation is 0.318874069609
the new corps of mcneillstown scored their first success of the season by winning

...

acoustic: 0.54139656
pianistic: 0.53954744
percussion: 0.5380066
looping: 0.5366536
microphone: 0.53594357
vocal: 0.5356803
sitar: 0.53254354
accordion: 0.5319634
acoustical: 0.5308091
fingerprint: 0.52998954
piano: 0.52979016
trumpet: 0.5286655
lyrical: 0.52858853
reverb: 0.52855355
synthesizer: 0.52781636
headlining: 0.52725923
orchestral: 0.526935
cello: 0.52684534
saxophone: 0.5267538
80-minute: 0.5266479
top_mutual_sim
0.20448866579681635
weight is 0.204488665797
the transposition of the soprano in b flat is the same as for b flat  ___  .
producing top 20 simwords
chords: 0.5469076
strings: 0.5394104
diamonds: 0.5376204
cadences: 0.53745425
sevenths: 0.53329307
recorders: 0.53054845
cellos: 0.5305365
basses: 0.5292569
harmonics: 0.5277783
semitones: 0.5268801
trills: 0.52587295
partials: 0.52562076
violins: 0.5239193
fingering: 0.5235371
tremolos: 0.52248263
sixths: 0.52220416
trumpets: 0.521901
clarinets: 0.52165115
chord: 0.5207077
bars: 0.52035004
top_mutual_sim
0.2359043504111

producing top 20 simwords
to-hand: 0.5540735
to-mouth: 0.5499022
in-hand: 0.5405869
to-eye: 0.53539217
washing: 0.52874535
wringing: 0.52839637
tooling: 0.5251267
cranking: 0.524771
stroking: 0.5245184
outs: 0.524166
stoking: 0.523119
wraps: 0.522157
painting: 0.52167845
welding: 0.5214978
craftsmanship: 0.52103025
stitching: 0.52029335
spinning: 0.5202686
swapping: 0.51962835
pelota: 0.51950264
printing: 0.5194042
top_mutual_sim
0.13673542812466621
weight is 0.136735428125
the set comes with a handy wooden ginger  ___  and 200 15 cm / 6 in wooden sate skewers .
producing top 20 simwords
sideboard: 0.5382283
stick: 0.53650874
knocker: 0.5360897
dresser: 0.5347632
bun: 0.5345741
cracker: 0.5336593
cutlet: 0.53335804
quilt: 0.5304626
twig: 0.53002197
pillow: 0.5292194
grater: 0.52905023
mug: 0.5278529
wrapper: 0.5278083
doll: 0.5274316
strainer: 0.52700895
basket: 0.5269162
sundae: 0.5268821
spatula: 0.526492
ladle: 0.52632415
scarf: 0.5263142
top_mutual_sim
0.16998089337721467
weight is

producing top 20 simwords
collectibles: 0.53832936
jewelry: 0.5350134
craftsmanship: 0.5316207
jewelery: 0.52864844
collectables: 0.52721703
art: 0.52691317
tailoring: 0.5266155
retailers: 0.5262479
shops: 0.5260867
goods: 0.5235464
cabinetry: 0.5235399
apparel: 0.52350897
dress: 0.52273464
jewellery: 0.52266246
packaging: 0.52213365
denim: 0.5221035
bookbinding: 0.5216963
clothing: 0.52043545
quality: 0.520362
diamonds: 0.5200853
top_mutual_sim
0.19811349734663963
weight is 0.198113497347
my fingertips looked as if they had been run down a cheese  ___  , and they stung on the sharp crimps .
producing top 20 simwords
crate: 0.54257137
tray: 0.5376478
shelf: 0.53230625
dispenser: 0.5321975
sprayer: 0.5321956
grater: 0.5309453
pot: 0.5303484
bug: 0.5299544
rack: 0.529895
platter: 0.5289713
sundae: 0.52783984
basket: 0.52745426
puff: 0.52705204
barrel: 0.5264202
grinder: 0.5264066
bed: 0.5258808
slide: 0.52525496
bowl: 0.52485925
cone: 0.5248446
precipice: 0.5246367
top_mutual_sim
0.15269

producing top 20 simwords
grain: 0.548173
sugar: 0.5467661
flour: 0.5467655
wheat: 0.5462115
rice: 0.54436815
salt: 0.54403996
bread: 0.5422548
vegetables: 0.5415982
nuts: 0.5413917
candles: 0.5403903
corns: 0.5395989
soap: 0.538912
firewood: 0.53890073
pith: 0.5388082
water: 0.5381679
manure: 0.5377524
wood: 0.5376696
wine: 0.53749925
crockery: 0.5371932
flowers: 0.53663397
top_mutual_sim
0.19334891717880964
weight is 0.193348917179
years abroad had made them grateful  ___  , bananas , cocoa beans and rice .
producing top 20 simwords
lemons: 0.53029084
growers: 0.5294804
acorns: 0.526123
asparagus: 0.5250652
potatoes: 0.52451915
belizeans: 0.52377653
bananas: 0.521712
dead: 0.5207915
radishes: 0.51997995
haitians: 0.51953375
entrees: 0.5189921
cowpeas: 0.51776797
cabbages: 0.5169597
givers: 0.5166876
pears: 0.5164945
cherries: 0.5158132
colonisers: 0.5157033
cereals: 0.5154846
salted: 0.51548433
flowers: 0.5154027
top_mutual_sim
0.19181647198274732
weight is 0.191816471983
grain whisk

toads: 0.54433864
snakes: 0.5381485
worm: 0.5377377
lizards: 0.5377175
grubs: 0.53599614
toad: 0.5343279
tadpole: 0.534159
frogs: 0.5336844
birds: 0.53072524
crocodiles: 0.5302849
lizard: 0.5302309
fish: 0.529865
tyrannosaurs: 0.5293851
pigs: 0.5293772
simian: 0.52920604
chickens: 0.5290819
snake: 0.5288941
cockroach: 0.5287674
cicada: 0.5283098
pythons: 0.5280965
top_mutual_sim
0.23833479173481464
weight is 0.238334791735
blackmer and culp are by now halfway across the swamp and have attracted the attention of several  ___  .
producing top 20 simwords
townspeople: 0.54863113
bystanders: 0.53751993
cops: 0.536946
hoodlums: 0.53452194
outsiders: 0.53245497
criminals: 0.5314023
others: 0.53134334
survivalists: 0.5309223
detectives: 0.5305654
townsfolk: 0.5305345
firemen: 0.53033286
paramedics: 0.5298749
policemen: 0.5292978
ranchers: 0.5286135
locals: 0.5285069
mobsters: 0.52849543
neighbors: 0.52829814
burglars: 0.5277559
patrons: 0.52742785
lawmen: 0.5263916
top_mutual_sim
0.2071469640